In [2]:
# 🐾 Install required packages (already installed, but kept for safety)
!pip install pandas matplotlib seaborn plotly

# 🐱 Upload your CSV files directly
from google.colab import files
uploaded = files.upload()

# 🧶 Load your data
import pandas as pd

# Check what files were uploaded
print("Uploaded files:")
for filename in uploaded.keys():
    print(f"- {filename}")

# 🐾 Load CSV data (adjust filenames if needed)
# Use the actual filenames from the uploaded list
df_dev = pd.read_csv('dev_predictions.csv')
df_test = pd.read_csv('test_predictions.csv')

# 🐈 Display first few rows to confirm loading
print("\nPreview of dev_predictions.csv:")
print(df_dev.head())

print("\nPreview of test_predictions.csv:")
print(df_test.head())


Saving test_predictions (3).csv to test_predictions (3) (1).csv
Saving dev_predictions (3).csv to dev_predictions (3) (1).csv
Saving dev_predictions (2).csv to dev_predictions (2) (1).csv
Saving test_predictions (2).csv to test_predictions (2) (1).csv
Saving test_predictions (1).csv to test_predictions (1) (1).csv
Saving dev_predictions (1).csv to dev_predictions (1) (1).csv
Saving test_predictions.csv to test_predictions (4).csv
Saving dev_predictions.csv to dev_predictions (4).csv
Uploaded files:
- test_predictions (3) (1).csv
- dev_predictions (3) (1).csv
- dev_predictions (2) (1).csv
- test_predictions (2) (1).csv
- test_predictions (1) (1).csv
- dev_predictions (1) (1).csv
- test_predictions (4).csv
- dev_predictions (4).csv

Preview of dev_predictions.csv:
  speaker_id                                          reference  \
0     KES004  it seems like some some fish or some seafood i...   
1     KES004  maybe it is kinda some milk or some some some ...   
2     KES004  it s a cake 

In [5]:
print(df_combined.columns.tolist())


['speaker_id', 'reference', 'prediction', 'wer', 'word_accuracy', 'lattescore_meaning_preserved', 'gender', 'age', 'severity_speech_impairment', 'type_nonstandard_speech', 'etiology', 'model', 'language']


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

# 🐾 Load your CSV files (adjust names to match what you uploaded)
df_small_en = pd.read_csv('dev_predictions.csv')
df_large_en = pd.read_csv('test_predictions.csv')
df_small_sw = pd.read_csv('dev_predictions.csv')
df_large_sw = pd.read_csv('test_predictions.csv')

# 🐱 Add model type and language columns
df_small_en['model'] = 'Small'
df_large_en['model'] = 'Large'
df_small_en['language'] = 'English'
df_large_en['language'] = 'English'

df_small_sw['model'] = 'Small'
df_large_sw['model'] = 'Large'
df_small_sw['language'] = 'Swahili'
df_large_sw['language'] = 'Swahili'

# 🧶 Combine datasets
df_combined = pd.concat([df_small_en, df_large_en, df_small_sw, df_large_sw], ignore_index=True)

# 1. 🐾 Overall Meaning Preservation by Language and Model
def plot_overall_meaning_preservation(df):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

    # Plot 1: Overall comparison by language and model
    overall_stats = df.groupby(['language', 'model'])['lattescore_meaning_preserved'].value_counts(normalize=True).unstack()
    overall_stats.plot(kind='bar', ax=ax1, color=['green', 'orange', 'red'])
    ax1.set_title('Meaning Preservation by Language and Model')
    ax1.set_ylabel('Percentage')
    ax1.legend(title='LATTES Meaning Levels')
    ax1.tick_params(axis='x', rotation=45)

    # Plot 2: Meaning preserved vs lost by language and model
    preserved_stats = df.groupby(['language', 'model']).apply(
        lambda x: (x['lattescore_meaning_preserved'] <= 1).mean(), include_groups=False
    ).unstack()
    preserved_stats.plot(kind='bar', ax=ax2, color=['blue', 'green', 'red', 'purple'])
    ax2.set_title('Percentage of Meaning Preserved (0+1) by Language and Model')
    ax2.set_ylabel('Percentage')
    ax2.legend(title='Model')
    ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

plot_overall_meaning_preservation(df_combined)

# 2. 🐾 Analysis by Severity
def plot_by_severity(df):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # English models by severity
    english_data = df[df['language'] == 'English']
    if not english_data.empty:
        severity_en = english_data.groupby(['model', 'severity_speech_impairment'])['lattescore_meaning_preserved'].mean().unstack()
        severity_en.T.plot(kind='bar', ax=ax1, color=['red', 'blue'])
        ax1.set_title('English: Average LATTES Score by Severity and Model (Lower = Better)')
        ax1.set_ylabel('Average LATTES Score')
        ax1.set_xlabel('Severity Level')
        ax1.legend(title='Model')
        ax1.tick_params(axis='x', rotation=45)

    # Swahili models by severity
    swahili_data = df[df['language'] == 'Swahili']
    if not swahili_data.empty:
        severity_sw = swahili_data.groupby(['model', 'severity_speech_impairment'])['lattescore_meaning_preserved'].mean().unstack()
        severity_sw.T.plot(kind='bar', ax=ax2, color=['red', 'blue'])
        ax2.set_title('Swahili: Average LATTES Score by Severity and Model (Lower = Better)')
        ax2.set_ylabel('Average LATTES Score')
        ax2.set_xlabel('Severity Level')
        ax2.legend(title='Model')
        ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

plot_by_severity(df_combined)

# 3. 🐾 Analysis by Etiology
def plot_by_etiology(df):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # English models by etiology
    english_data = df[df['language'] == 'English']
    if not english_data.empty:
        etiology_en = english_data.groupby(['model', 'etiology'])['lattescore_meaning_preserved'].mean().unstack()
        etiology_en.T.plot(kind='bar', ax=ax1, color=['red', 'blue'])
        ax1.set_title('English: Average LATTES Score by Etiology and Model (Lower = Better)')
        ax1.set_ylabel('Average LATTES Score')
        ax1.set_xlabel('Etiology')
        ax1.legend(title='Model')
        ax1.tick_params(axis='x', rotation=45)

    # Swahili models by etiology
    swahili_data = df[df['language'] == 'Swahili']
    if not swahili_data.empty:
        etiology_sw = swahili_data.groupby(['model', 'etiology'])['lattescore_meaning_preserved'].mean().unstack()
        etiology_sw.T.plot(kind='bar', ax=ax2, color=['red', 'blue'])
        ax2.set_title('Swahili: Average LATTES Score by Etiology and Model (Lower = Better)')
        ax2.set_ylabel('Average LATTES Score')
        ax2.set_xlabel('Etiology')
        ax2.legend(title='Model')
        ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

plot_by_etiology(df_combined)


FileNotFoundError: [Errno 2] No such file or directory: 'dev_predictions.csv'